Step 1: Preparation 

Before you begin, make sure that you have set the runtime type to GPU  (Hardware acclerator: GPU).

In [1]:
import sys
sys.path.append("ailab/Real-CUGAN")
ROOTPATH="./" # root dir (a constant)
ModelPath=ROOTPATH+"weights_v3/" # model dir
PendingPath=ROOTPATH+"pending/" # input dir
FinishPath=ROOTPATH+"finish/" # output dir
ModelName="pro-conservative-up2x.pth" # default model

Step 2: Download model files from [here](https://drive.google.com/drive/folders/1UFgpV14uEAcgYvVw0fJuajzy1k7JIz6H) and save them - in a folder called `weights_v3` 

Step 3: Put/upload your image(s) under `pending`.

Then, run the following and choose the model you want.

In [2]:
import glob,os
fileNames = os.listdir(PendingPath)
print("Pending images:")
for i in fileNames:
    print("\t"+i)
fileNames = glob.glob(ModelPath+"/*.pth")
print("Model files available:")
models=[]
for idx, i in enumerate(fileNames):
    print(f"{idx+1}. \t {os.path.split(i)[1]}")
    models.append(os.path.split(i)[1])

Pending images:
	632d634919cab145f2372a3196665957~tplv-dy-resize-origshort-autoq-75_330.jpeg
Model files available:
1. 	 pro-conservative-up2x.pth
2. 	 pro-denoise3x-up3x.pth
3. 	 pro-conservative-up3x.pth
4. 	 pro-denoise3x-up2x.pth
5. 	 pro-no-denoise-up3x.pth
6. 	 pro-no-denoise-up2x.pth
7. 	 up3x-latest-conservative.pth
8. 	 up2x-latest-no-denoise.pth
9. 	 up2x-latest-denoise3x.pth
10. 	 up3x-latest-denoise3x.pth
11. 	 up3x-latest-no-denoise.pth
12. 	 up4x-latest-conservative.pth
13. 	 up4x-latest-denoise3x.pth
14. 	 up2x-latest-denoise1x.pth
15. 	 up4x-latest-no-denoise.pth
16. 	 up2x-latest-conservative.pth
17. 	 up2x-latest-denoise2x.pth


Step 4: Run the processing script.

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
import torch
from torch import nn as nn
from torch.nn import functional as F
import numpy as np
from upcunet_v3 import RealWaifuUpScaler
from time import time as ttime
from torch import nn
fileNames = os.listdir(PendingPath)
class M2(nn.Module):
    def __init__(self,m,pro):
        super().__init__()
        self.m=m
        self.pro=pro
    def forward(self,x):
        x=torch.permute(x,(0,3,1,2))
        if self.pro:
            x=x.half() / (255/0.7)+0.15
        else:
            x=x.half() / 255
        return torch.permute(m.forward_fast_rough(x,0,1,True),(0,2,3,1))
for ModelName in models:
    print(f"using model {ModelPath+ModelName}")
    scale=int(ModelName[ModelName.find("up")+2])
    upscaler = RealWaifuUpScaler(scale, ModelPath+ModelName, half=True, device="cuda:0")
    pro=upscaler.pro
    m=upscaler.model
    m2=M2(m,pro)
    dummy_input = torch.randn(1,  224, 224,3, device="cuda")
    input_names=["img"]
    output_names=["out"]
    dynamic = {'img': {0: 'batch', 1: 'height', 2: 'width'}}
    dynamic['out'] = {0: 'batch', 1: 'out_height', 2: 'out_width'}
    torch.onnx.export(m2, dummy_input, ModelName+".onnx", verbose=False, 
                  input_names=input_names, 
                  output_names=output_names,
                  opset_version=15,
                  dynamic_axes=dynamic or None)

using model ./weights_v3/pro-conservative-up2x.pth


/usr/src/ailab/Real-CUGAN/upcunet_v3.py:267: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if (w0 != pw or h0 != ph): x = x[:, :, :h0 * 2, :w0 * 2]
/opt/conda/lib/python3.10/site-packages/torch/onnx/_internal/jit_utils.py:307: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392020201/work/torch/csrc/jit/passes/onnx/constant_fold.cpp:179.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
/opt/conda/lib/python3.10/site-packages/torch/onnx/utils.py:702: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally a

using model ./weights_v3/pro-denoise3x-up3x.pth


/usr/src/ailab/Real-CUGAN/upcunet_v3.py:595: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if (w0 != pw or h0 != ph): x = x[:, :, :h0 * 3, :w0 * 3]


using model ./weights_v3/pro-conservative-up3x.pth
using model ./weights_v3/pro-denoise3x-up2x.pth
using model ./weights_v3/pro-no-denoise-up3x.pth
using model ./weights_v3/pro-no-denoise-up2x.pth
using model ./weights_v3/up3x-latest-conservative.pth
using model ./weights_v3/up2x-latest-no-denoise.pth
using model ./weights_v3/up2x-latest-denoise3x.pth
using model ./weights_v3/up3x-latest-denoise3x.pth
using model ./weights_v3/up3x-latest-no-denoise.pth
using model ./weights_v3/up4x-latest-conservative.pth


/usr/src/ailab/Real-CUGAN/upcunet_v3.py:926: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if (w0 != pw or h0 != ph): x = x[:, :, :h0 * 4, :w0 * 4]


using model ./weights_v3/up4x-latest-denoise3x.pth
using model ./weights_v3/up2x-latest-denoise1x.pth
using model ./weights_v3/up4x-latest-no-denoise.pth
using model ./weights_v3/up2x-latest-conservative.pth
using model ./weights_v3/up2x-latest-denoise2x.pth


Generated images should be under /content/aliab/Real-CUGAN/finish.

In [21]:

m2=M2(m)
import torch
dummy_input = torch.randn(1,  224, 224,3, device="cuda")


input_names=["img"]
output_names=["out"]
dynamic = {'img': {0: 'batch', 1: 'height', 2: 'width'}}
dynamic['out'] = {0: 'batch', 2: 'out_height', 3: 'out_width'}
torch.onnx.export(m2, dummy_input, ModelName+".onnx", verbose=False, 
                  input_names=input_names, 
                  output_names=output_names,
                  opset_version=15,
                  dynamic_axes=dynamic or None)